# Modèle 1 : Implementation sentence transformers étude complementaire (Architecture 2)

Dans l'étude précédente nous nous avons fais ces choix :

- **Sens sémentique de la requête de l'utilisateur :** Nous calculons un vecteur pour notre domaine d'activité ainsi que pour chacune des phrases de notre description. Ensuite nous faisons la moyenne de ces vecteurs afin d'avoir un vecteur qui représente le sens sémentique globale de notre requête.

- **Sens sémentique des solutions :** Pour chaque solution nous avons fais pareil, nous calculons un vecteur pour chaque phrases et nous faisons la moyenne. L'idée était d'avoir le sens sémentique de la solutions au globale.

**Motivation pour cette étude complémentaire :** En tant qu'humain, lorsque nous faisons une recherche nous avons tendances à lire en diagonale les documents afin de trouver une partie pertinente. Cette révélation nous a fait remarquer qu'il pourrait être pertinent de comparer chaque phrase indépendament avec l'entrée de l'utilisateur et ne garder que le max de la correspondance sémentique. Cependant, nous gardons le même sens sémentique pour la reqûete de l'utilisateur car nous souhaitons lui proposer des solutions qui correspondent parfaitement à sa requête.

## Imports

In [1]:
!pip install pandas spacy bs4 sentence_transformers numpy
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 3.5 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [1]:
import pandas as pd
import re
import unicodedata
import spacy
from bs4 import BeautifulSoup
from spacy.lang.fr.stop_words import STOP_WORDS
import pickle
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.quantization import quantize_embeddings
import numpy as np

/home/patrice/Documents/Etudes/CY-Tech/Ing2_Sem2_2023-2024/UE7_Projet_DataBattle_IA-Pau/ia-pau-data-battle-mars-2024/backend/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Fonctions inchangées de l'ancienne étude

In [2]:
def load_and_merge_data(csv_file = '../data/solutions.csv'):
    # Charger le fichier CSV en spécifiant le séparateur '|'
    df = pd.read_csv(csv_file, sep='|', header=None)
    # Renommer les colonnes
    df.columns = ['id_solution', 'categorie', 'texte']
    # Filtrer les lignes pour les catégories spécifiées
    categories_specifiees = [1, 2, 5, 6]
    df_filtre = df[df['categorie'].isin(categories_specifiees)]
    # Pivoter les données pour obtenir les colonnes 'titre', 'definition', 'application' et 'bilan énergie'
    df_pivot = df_filtre.pivot(index='id_solution', columns='categorie', values='texte').reset_index()
    # Renommer les colonnes
    df_pivot.columns = ['id_solution', 'titre', 'definition', 'application', 'bilan_energie']

    # Ajouter un point à la fin des colonnes si nécessaire
    colonnes = ['titre', 'definition', 'application', 'bilan_energie']
    for col in colonnes:
        df_pivot[col] = df_pivot[col].apply(lambda x: x.strip() + '.' if isinstance(x, str) and x.strip()[-1] != '.' else x.strip() if isinstance(x, str) else x)
    
    # Gérer les valeurs NaN lors de la fusion des colonnes
    def combine_text(row):
        text_parts = [row[col] for col in colonnes if pd.notnull(row[col])]
        return ' '.join(text_parts)
    
    # Appliquer la fonction pour créer la nouvelle colonne 'text'
    df_pivot['text'] = df_pivot.apply(combine_text, axis=1)

    # Sélectionner uniquement les colonnes 'id_solution' et 'text'
    df_final = df_pivot[['id_solution', 'text']]
    
    return df_final

def pre_processing(texte):
    # Nettoyer HTML Tags
    texte = BeautifulSoup(texte, 'html.parser').get_text()

    # Remplacer "&nbsp;." par rien
    texte = re.sub(r'&nbsp;\.', '', texte)

    # Accents
    texte = unicodedata.normalize('NFD', texte).encode('ascii', 'ignore').decode("utf-8")

    # Retirer les numéros
    texte = re.sub(r'\b\d+\b', '', texte)

    # Tokenization, Lemmatization, Removing Stopwords, Lowercase
    doc = nlp(texte)
    phrases_propres = []
    for phrase in doc.sents:
        tokens = [token.lemma_.lower() for token in phrase if not token.is_stop and not token.is_punct and not token.is_space]
        phrase_propre = ' '.join(tokens)
        if phrase_propre:
            phrases_propres.append(phrase_propre + ".")  # Ajouter un point à la fin de la phrase propre

    # Joining the cleaned sentences back into a single string
    cleaned_text = ' '.join(phrases_propres)

    return cleaned_text

def calculate_average_embedding(text, quantize=False, precision="binary"):
    # Diviser le texte en phrases
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]
    
    # Calculer l'embedding de chaque phrase
    if quantize :
        sentence_embeddings = model.encode(sentences, precision=precision)
    else :
        sentence_embeddings = model.encode(sentences)
    
    # Prendre la moyenne des embeddings des phrases
    if len(sentence_embeddings) > 0:
        average_embedding = np.mean(sentence_embeddings, axis=0)
    else:
        average_embedding = np.zeros(model.get_sentence_embedding_dimension())
    
    return average_embedding

## Fonctions adaptées

In [3]:
# NOUVELLE FONCTION
def calculate_sentence_embedding(text, quantize=False, precision="binary"):
    # Diviser le texte en phrases
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]
    
    # Calculer l'embedding de chaque phrase
    if quantize :
        sentence_embeddings = model.encode(sentences, precision=precision)
    else :
        sentence_embeddings = model.encode(sentences)
    
    return sentence_embeddings

In [5]:
def genere_embedding(data, output_file, quantize=False, precision="binary"):

    # Appliquer la fonction pour calculer l'embedding moyen à chaque texte
    if quantize:
        embeddings = data['clean_text'].apply(calculate_sentence_embedding, quantize=True)
    else :
        embeddings = data['clean_text'].apply(calculate_sentence_embedding)

    
    # Créer un nouveau DataFrame avec id_solution et les embeddings
    new_data = {
        'id_solution': data['id_solution'],
        'text_embedding': embeddings # Désormais mes embeddings sont des listes de solutions contenant chacune les vecteurs de toutes les phrases de la solution.
    }
    
    # Créer un DataFrame à partir des nouvelles données
    df_embeddings = pd.DataFrame(new_data)
    
    # Storer les embeddings dans un fichier
    with open(output_file, "wb") as fOut:
        pickle.dump(df_embeddings, fOut, protocol=pickle.HIGHEST_PROTOCOL)
    
    print("Les embeddings ont été storer avec succès.")

In [6]:
def find_solution(text_to_compare, embeddings_file, quantize=False, precision="binary"):
    # Calculer l'embedding moyen du texte à comparer
    embedding_to_compare = calculate_average_embedding(text_to_compare, quantize, precision)
    
    # Charger les embeddings à partir du fichier
    with open(embeddings_file, "rb") as fIn:
        df_embeddings = pickle.load(fIn)


    list_similarities = []
    # Pour chaque solution
    for solution in df_embeddings['text_embedding'].values:
        embeddings_array = np.stack(solution)
        # Calculer la similarité cosinus entre l'embedding à comparer et les embeddings dans df_embeddings
        similarities = util.pytorch_cos_sim(embedding_to_compare.reshape(1, -1), embeddings_array)

        max_value, _ = similarities.max(dim=1)
        list_similarities.append(max_value)

    # Ajouter les similarités au DataFrame df_embeddings
    df_embeddings['similarity'] = list_similarities
    
    # Trier par similarité décroissante
    df_sorted = df_embeddings.sort_values(by='similarity', ascending=False)
    
    # Récupérer les id_solution et les similarités
    solution_info = df_sorted[['id_solution', 'similarity']].head(10)
    
    # Convertir en liste de tuples (id_solution, similarity)
    solution_list = list(zip(solution_info['id_solution'], solution_info['similarity']))
    
    return solution_list

## Tests

In [7]:
df_solutions = load_and_merge_data()
# Charger le modèle spaCy pour le français
nlp = spacy.load("fr_core_news_sm")
# Appliquons le traitement à la colonne text de notre df_solutions
df_solutions['clean_text'] = df_solutions['text'].apply(pre_processing)
# Charger le modèle
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

/tmp/ipykernel_49969/2412020569.py:34: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  texte = BeautifulSoup(texte, 'html.parser').get_text()


In [10]:
print(df_solutions)

      id_solution                                               text  \
0               2  Installation frigorifique négative de type cas...   
1               3  Centrale négative en mode booster. Lorsque l'i...   
2               4  Arrêt des compresseurs le week end et lorsque ...   
3               5  Régulation non électronique sur un compresseur...   
4               6  Condenseur frigorifique à haute efficacité. Mi...   
...           ...                                                ...   
1095         1710  Favoriser la communication digitale au catalog...   
1096         1711  Remplacer le chauffage des bains au gaz par un...   
1097         1712  Mettre en place des portes sectionnelles pour ...   
1098         1713          Films anti-chaleur sur les baies vitrées.   
1099         1714  Optimiser la production d'eau traitée (osmosé,...   

                                             clean_text  
0     installation frigorifique negativ type cascade...  
1     centrale nega

Génère embedding

In [10]:
# Générons notre embedding
genere_embedding(df_solutions, "embeddings/Architecture-2_paraphrase-multilingual-mpnet-base-v2_embeddings.pkl")

Les embeddings ont été storer avec succès.


Test inférence

In [8]:
# Malheureusement nous n'avons pas reçu le domaine d'activité correspondant aux 
# requetes avec Kerdos.Et pour certaines requetes nous n'avons pas l'Id_solution non
# plus, dans ce cas il est remplacé par -1.
dataset_test_kerdos = [
    ["Id_solution", "Domaine_activite", "Description"],
    [54, "", "Moteur à haut rendement"], # Titre de sol 54
    [156, "", "Remplacement des outils pneumatiques par des outils électriques lorsque c'est possible "], # Titre de sol 156
    [724, "","C'est quoi la HP flottante ?"],
    [914, "", "Je voudrais dimensionner un panneau solaire."],
    [719, "", "Quel gain pour un variateur de vitesse ?"],
    [-1, "", "Comment faire pour réduire la consommation de mon compresseur d'air comprimé ?"],
    [-1, "", "J'aimerais avoir une régulation optimisée de mon groupe froid"],
    [-1, "", "Quelles sont les meilleures solutions pour l'agro-alimentaire ?"],
    [-1, "", "Comment faire pour réduire la consommation de mon compresseur d'air comprimé ?"]
]

In [9]:
# Fonction appelé par notre utilisateur
def model_PAT(secteur, description, embedding_paths) :
    # On commence par concaténer notre secteur et notre description.
    text = secteur + ". " + description

    # Ensuite on applique notre pré-processing
    clean_text = pre_processing(text)

    # Ensuite on cherche nos similarités 
    solutions = find_solution(clean_text, embedding_paths)

    # On return une liste contenant uniquement le numéros des solutions
    id_solutions = []
    for solution in solutions :
        id_solutions.append(solution[0])

    return id_solutions

In [14]:
embedding_paths = "embeddings/Architecture-2_paraphrase-multilingual-mpnet-base-v2_embeddings.pkl"
# On va tester sur notre dataset_test
for i in range(1,len(dataset_test_kerdos)):
    print("--------------------------------------------")
    print("Solution attendue : ", dataset_test_kerdos[i][0])
    print(model_PAT(dataset_test_kerdos[i][1], dataset_test_kerdos[i][2]))

--------------------------------------------
Solution attendue :  54
[328, 789, 58, 51, 504, 1014, 1560, 932, 50, 243]
--------------------------------------------
Solution attendue :  156
[156, 157, 1050, 58, 32, 77, 853, 777, 1022, 267]
--------------------------------------------
Solution attendue :  724
[1129, 1533, 388, 823, 294, 229, 531, 855, 386, 63]
--------------------------------------------
Solution attendue :  914
[1454, 1452, 1453, 1457, 1455, 1456, 1486, 818, 20, 769]
--------------------------------------------
Solution attendue :  719
[63, 1142, 1144, 794, 1014, 941, 1140, 58, 146, 1495]
--------------------------------------------
Solution attendue :  -1
[162, 1001, 332, 1051, 1058, 159, 4, 157, 171, 93]
--------------------------------------------
Solution attendue :  -1
[12, 191, 483, 103, 148, 790, 803, 10, 42, 171]
--------------------------------------------
Solution attendue :  -1
[1560, 268, 739, 259, 1513, 991, 544, 276, 562, 1105]
----------------------------

# TEST FINAL - Architecture 2 : 

Test_set : patrice_test_set

In [10]:
df_solutions = load_and_merge_data()
# Charger le modèle spaCy pour le français
nlp = spacy.load("fr_core_news_sm")
# Appliquons le traitement à la colonne text de notre df_solutions
df_solutions['clean_text'] = df_solutions['text'].apply(pre_processing)

/tmp/ipykernel_49969/2412020569.py:34: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  texte = BeautifulSoup(texte, 'html.parser').get_text()


In [11]:
# Lire le fichier CSV
df_testset = pd.read_csv("data/patrice_test_set.csv")

def test_accuracy(embedding_paths, dataset=df_testset, top_n=1) :
    accuracy = 0
    for i in range(1,len(dataset)):
        predictions = model_PAT("", dataset['Description'][i], embedding_path)
        if (dataset["id_solution"][i] in predictions[:top_n]):
            accuracy += 1/len(dataset)
        else :
            print("mal prédit : ", dataset["id_solution"][i])
    return accuracy

## Modèle : paraphrase-multilingual-mpnet-base-v2_embeddings.pkl

In [12]:
# Charger le modèle
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

In [13]:
# Générons notre embedding
genere_embedding(df_solutions, "embeddings/Architecture-2_paraphrase-multilingual-mpnet-base-v2_embeddings.pkl")

In [ ]:
test_accuracy("embeddings/Architecture-2_paraphrase-multilingual-mpnet-base-v2_embeddings.pkl")

## Modèle : sentence-transformers/LaBSE

In [ ]:
# Charger le modèle
model = SentenceTransformer("sentence-transformers-LaBSE")

In [ ]:
# Générons notre embedding
genere_embedding(df_solutions, "embeddings/Architecture-2_sentence-transformers-LaBSE.pkl")

In [ ]:
test_accuracy("embeddings/Architecture-2_sentence-transformers-LaBSE.pkl")